# 🧬 Geometry-Complete Equivariant Diffusion
## De Novo Drug Design Training

**All processing on LOCAL Colab disk (fast!)**

**Checkpoints saved to Drive (persistent)**

## Cell 1: Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
print(f'GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None!"}')

%pip install -q torch-geometric rdkit scipy numpy pyyaml tqdm scikit-learn
print('✅ Setup complete')

## Cell 2: Clone Repo

In [ ]:
import os
REPO = '/content/drive/MyDrive/geom_diffusion'
if not os.path.exists(REPO):
    !git clone https://github.com/Nethrananda21/geom_diffusion.git {REPO}
%cd {REPO}
!git pull origin master

## Cell 3: Download Pre-processed Data (Zenodo)

**crossdocked_pocket10** - 1.6GB preprocessed dataset from PMDM paper

In [ ]:
import os

DATA_DIR = '/content/data'
os.makedirs(DATA_DIR, exist_ok=True)

# Download from Zenodo (verified working)
if not os.path.exists(f'{DATA_DIR}/crossdocked_pocket10'):
    print('📥 Downloading crossdocked_pocket10 from Zenodo (1.6GB)...')
    !wget -q --show-progress -O {DATA_DIR}/crossdocked_pocket10.tar.gz \
        https://zenodo.org/record/10519710/files/crossdocked_pocket10.tar.gz
    
    print('📦 Extracting...')
    !tar -xzf {DATA_DIR}/crossdocked_pocket10.tar.gz -C {DATA_DIR}/
    !rm {DATA_DIR}/crossdocked_pocket10.tar.gz
    print('✅ Done!')
else:
    print('✅ Data already exists')

!ls -la {DATA_DIR}/

## Cell 4: Inspect Data Structure

In [ ]:
import os
import pickle

DATA_DIR = '/content/data/crossdocked_pocket10'

# List available files
print('📁 Files in dataset:')
for f in os.listdir(DATA_DIR):
    size = os.path.getsize(f'{DATA_DIR}/{f}') / 1e6
    print(f'  {f}: {size:.1f} MB')

# Check for index.pkl
index_file = f'{DATA_DIR}/index.pkl'
if os.path.exists(index_file):
    with open(index_file, 'rb') as f:
        index = pickle.load(f)
    print(f'\n📊 Index: {len(index)} samples')
    print(f'   Sample keys: {list(index[0].keys()) if hasattr(index[0], "keys") else type(index[0])}')

## Cell 5: Create Stratified 5K Subset

Size stratification + diversity + no data leakage

In [ ]:
import pickle
import numpy as np
from collections import defaultdict
from sklearn.cluster import KMeans
import os

np.random.seed(42)

DATA_DIR = '/content/data/crossdocked_pocket10'
OUT_DIR = '/content/data/crossdocked'
os.makedirs(OUT_DIR, exist_ok=True)

# Load index
with open(f'{DATA_DIR}/index.pkl', 'rb') as f:
    full_data = pickle.load(f)
print(f'Loaded {len(full_data)} samples')

# Group by pocket
pockets = defaultdict(list)
pocket_info = {}

for i, sample in enumerate(full_data):
    if isinstance(sample, dict):
        pocket_id = sample.get('pocket_id', sample.get('protein_filename', f'pocket_{i}'))
        pocket_coords = np.array(sample.get('protein_pos', sample.get('pocket_coords', [])))
    else:
        pocket_id = str(i)
        pocket_coords = np.zeros((1, 3))
    
    pockets[pocket_id].append(sample)
    if pocket_id not in pocket_info:
        pocket_info[pocket_id] = {
            'size': len(pocket_coords),
            'centroid': pocket_coords.mean(axis=0) if len(pocket_coords) > 0 else np.zeros(3)
        }

print(f'Found {len(pockets)} unique pockets')

# Filter by size and ligand count
MAX_SIZE, MIN_LIGANDS = 250, 10  # Relaxed for this dataset
valid = [p for p, s in pockets.items() 
         if pocket_info[p]['size'] <= MAX_SIZE and len(s) >= MIN_LIGANDS]
print(f'After filter: {len(valid)} pockets')

# Stratify by size
small = [p for p in valid if pocket_info[p]['size'] <= 100]
medium = [p for p in valid if 100 < pocket_info[p]['size'] <= 175]
large = [p for p in valid if 175 < pocket_info[p]['size'] <= 250]

print(f'Bins: Small={len(small)}, Medium={len(medium)}, Large={len(large)}')

# Select diverse pockets using K-means
def select_diverse(plist, n):
    if len(plist) <= n: return plist
    centroids = np.array([pocket_info[p]['centroid'] for p in plist])
    km = KMeans(n_clusters=min(n, len(plist)), random_state=42, n_init=10).fit(centroids)
    return [plist[[i for i in range(len(plist)) if km.labels_[i] == c][0]] for c in range(min(n, len(plist))) if any(km.labels_ == c)]

# 40 from each bin
sel_small = select_diverse(small, min(40, len(small)))
sel_medium = select_diverse(medium, min(40, len(medium)))
sel_large = select_diverse(large, min(40, len(large)))

# Stratified split (preserve ratios)
np.random.shuffle(sel_small); np.random.shuffle(sel_medium); np.random.shuffle(sel_large)

n_train_small = int(len(sel_small) * 0.83)
n_train_medium = int(len(sel_medium) * 0.83)
n_train_large = int(len(sel_large) * 0.83)

train_pockets = sel_small[:n_train_small] + sel_medium[:n_train_medium] + sel_large[:n_train_large]
val_pockets = sel_small[n_train_small:] + sel_medium[n_train_medium:] + sel_large[n_train_large:]

print(f'\n✅ Train: {len(train_pockets)} pockets, Val: {len(val_pockets)} pockets')
print(f'✅ Overlap: {len(set(train_pockets) & set(val_pockets))}')

# Create datasets (up to 50 ligands per pocket)
train_samples = [s for p in train_pockets for s in pockets[p][:50]]
val_samples = [s for p in val_pockets for s in pockets[p][:50]]

print(f'📊 Train: {len(train_samples)}, Val: {len(val_samples)}')

# Save
with open(f'{OUT_DIR}/train_data.pkl', 'wb') as f:
    pickle.dump(train_samples, f)
with open(f'{OUT_DIR}/val_data.pkl', 'wb') as f:
    pickle.dump(val_samples, f)

print('💾 Saved to /content/data/crossdocked/')

## Cell 6: Update Config

In [ ]:
import yaml

%cd /content/drive/MyDrive/geom_diffusion

with open('configs/debug_t4.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

cfg['data']['root'] = '/content/data'
cfg['data']['train_file'] = 'crossdocked/train_data.pkl'
cfg['data']['val_file'] = 'crossdocked/val_data.pkl'
cfg['training']['max_epochs'] = 50
cfg['training']['batch_size'] = 4
cfg['hardware']['num_workers'] = 2

with open('configs/debug_t4.yaml', 'w') as f:
    yaml.dump(cfg, f)

print('✅ Config updated')

## Cell 7: Delete Cache & Train 🚀

In [ ]:
import shutil
from pathlib import Path

%cd /content/drive/MyDrive/geom_diffusion

# Delete cache
for cache in ['/content/data/cache', 'data/cache']:
    if Path(cache).exists():
        shutil.rmtree(cache)
        print(f'🗑️ Deleted {cache}')

# Train
!python train.py --config configs/debug_t4.yaml --checkpoint_dir checkpoints

## Cell 8: Resume Training

In [ ]:
# After disconnect: Run Cells 1, 2, 6 then this:
# %cd /content/drive/MyDrive/geom_diffusion
# !python train.py --config configs/debug_t4.yaml --resume checkpoints/best_model.pt